### **STEP-1**. 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. ALO 의 main.py 파일이 존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> conda create -n {ENV-NAME} python=3.10 \
> conda init bash \
> conda activate {ENV-NAME} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}

2. 본 jupyter notebook 에서, 위에서 생성한 ipykernel 을 선택 합니다. \
   가령 tcr이라는 이름의 가상환경을 만들었다면, 아래와 같이 선택합니다.

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>


----

### **STEP-2**. Solution 정보 입력 및 Infra 정보 입력

##### Solution information (dict) 작성 
- pipeline_type: 파이프라인이 학습 및 추론 과정을 지원하는 표시     
  - ex1: 학습 과 추론 모두 지원 --> ['train', 'inference]     
  - ex2: 추론만 지원 --> ['inference']    
  - ex3: 학습 만은 미지원 --> ~~['train']~~ 

- solution_update: 기존 솔루션을 업데이트 할지 결정
  - True: 업데이트 진행. 기존 name 과 동일한 이름 입력 (name 이 존재하지 않으면 에러)  
  - False: 신규 생성. 기존 name 과 다르게 입력 (name 이 존재하면 에러)

- name: 솔루션 이름 
  - 주의사항: 스페이스, 특수 문자, 한글은 미지원 

- description: 솔루션이 Edge Conductor 의 UI 로 표시 될 때, 설명 가이드 작성 
  - overview: 솔루션에 대한 전반적을 설명을 작성. (markdown 을 지원)
  - 고려사항: overview 외에 다양한 sub-title 지원 예정.

- contents_type: re-train, re-labeling 을 위한 설명 
  - support_labeling: re-labeling 할지 결정. True 일 경우,   
    Edge Conductor 에서 re-labeling 기능 활성화.   
  - inference_result_datatype: EdgeApp 에 inference 결과를 표시하는 방법으로 'table', 'image' 중에 하나 선택. 
    AI Contents 제작 시, output.csv, output.jpg 를 생성해 두어야 함
  - train_datatype: re-train 에 사용될 데이터 포멧 결정으로 'table', 'image' 중에 하나 선택. 
    AI Contents 제작 시, output.csv, output.jpg 를 생성해 두어야 함

- train_gpu: True, False 중에 선택. True 일 경우, gpu 용 train docker container 제작 
- train_gpu: True, False 중에 선택. True 일 경우, gpu 용 inference docker container 제작 
- inference_arm: True, False 중에 선택. True 일 경우, arm 용 inference docker container 제작


    

In [201]:
#----------------------------------------#
#        AI Solution Spec 작성           #
#----------------------------------------#
solution_info ={
    'pipeline_type': ["train", "inference"], # "single", "multi"
    'solution_update': True,
    'name': 'tcr-test-ssh',
    'description': {
        "overview": "Test TCR"
    },
    'contents_type': {
            'support_labeling': False,
            'inference_result_datatype': 'table', # 'image'
            'train_datatype': 'table' # 'image'
    },
    'train_gpu': True, ## cpu, gpu
    'inference_gpu': True,
    "inference_arm": True  # amd, arm  
}

##### AI Solution 이 동작될 Infra (dict) 설정
- AIC_URI: AI Conductor 또는 Solution Storage 의 web server 주소. 
  - 고객지수플랫폼: "https://aic-kic.aidxlge.com/"
  - 마그나 운영과제: "https://aic-kic.aidxlge.com/"
  - 담당테스트 환경: "http://10.158.2.243:9999/"
  - 사외테스트 환경: "https://web.aic-dev.lgebigdata.com/" 

- REGION: AWS Cloud 의 region 값 ("ap-northeat-2" 고정)
- WORKSPACE_NAME: 프로젝트 명을 의미.   
  (기술적의미: ML Ochestractor 의 물리적 태그 명으로 Cost 측정 단위)
  - 고객지수플랫폼: "cism-ws"
  - 마그나 운영과제: "magna-ws"
  - 담당테스트 환경: "cism-ws", "magna-ws"
  - 사외서비스 환경: "cism-ws", "magna-ws" 

- BUILD_METHOD: docker build 하는 방식 설정.    
  docker container 형태의 jupyter notebook 에서 솔루션 제작 시,     
  buildah (빌다) 라는 제작 방식을 사용해야 함.     
  - 고객지수플랫폼: "buildah"
  - 마그나운영과제: "buildah"
  - 담당테스트 환경 (솔루션 제작은 로컬서버 jupyter 실행): "docker"
  - 사외서비스 환경 (솔루션 제작은 로컬서버 jupyter 실행): "docker"

- VERSION: AI Solution 의 metadata 인 solution_metadata 의 format 버전.   
           AIC_URI 마다 지원버전이 다름.    
  - 모든 플랫폼 & 서비스 환경: 1.0 으로 동일 

- SOLUTION_TYPE: AI Solution 이 공유되는 계정 범위. 'private', 'public' 중에 하나 선택.   
  - 'public' : AI-Advisor 가 설치될 때, built-in 되어 설치 할 solution 의미.   
    (SYSTEM MANAGER 만 등록 가능)
  - 'private': 과제 단위로 공유 될 solution. 

- REPOSITORY_TAGS: ECR repository 를 생성 시, 입력 할 Tags 값. 과금를 위한 표시 정보 (buildah 모드만 지원)
  - 고객지수플랫폼, 마그나운영과제:    
    "REPOSITORY_TAGS": [ 
      "Key=Company,Value=LGE",    
      "Key=Owner,Value=IC360",    
      "Key=HQ,Value=CDO",    
      "Key=Division,Value=CDO",    
      "Key=Infra Region,Value=KIC",    
      "Key=Service Mode,Value=OP",     
      "Key=Cost Type,Value=COMPUTING",     
      "Key=Project,Value=CIS",     
      "Key=Sub Project,Value=CISM",     
      "Key=System,Value=AIDX"]     

In [146]:
infra_setup = {
    ### system info. 
    'AIC_URI': "https://web.aic-dev.lgebigdata.com/",  ## 실 서버 
    "REGION": "ap-northeast-2",
    "WORKSPACE_NAME": "cism-ws",  ## magna-ws, cism-ws
    "BUILD_METHOD": "docker",  ## docker, buildah 

    'VERSION': 1.0,
    "LOGIN_MODE": "static", ## ldap, static
    "ECR_TAG": "latest",   ## 없어질 사항
    "SOLUTION_TYPE": "private", ##public, private 
    "REPOSITORY_TAGS": [ 
      "Key=Company,Value=LGE",
      "Key=Owner,Value=IC360",
      "Key=HQ,Value=CDO",
      "Key=Division,Value=CDO",
      "Key=Infra Region,Value=KIC",
      "Key=Service Mode,Value=OP",
      "Key=Cost Type,Value=COMPUTING",
      "Key=Project,Value=CIS",
      "Key=Sub Project,Value=CISM",
      "Key=System,Value=AIDX"]
}



### **STEP-3**. AI Solution 등록

##### 3-1. Username & Password 를 입력 

> !! 접속이 실패되는 경우, AI Conductor 에게 계정 생성을 요청 하시기 바랍니다.   
> Contact Us: hyunsoo0802.lim@lge.com  (LGE AI빅데이터담당, AI Conductor 담당자)

> Login 방식이 {'LOGIN_METHOD': 'ldap'} 로 설정되어 있는 경우, LGEP 의 ID & PW 를 사용


In [154]:
import getpass

username = input('Username: ')
password = getpass.getpass('Password: ')

print("Your ID : ", username)
print("Your PW : ", password)

Your ID :  cism-dev
Your PW :  cism-dev@com


##### 3-2. ALO 솔루션 등록 실행
아래와 같은 과정이 순차적으로 실행 됨. 진행 중 Error 가 발생하게 되면, STEP-2 설정을 변경 함.
> Contact US: sehyun.song@lge.com (LGE AI빅데이터 담당, ALO 담당자)

AI Solution 등록 과정 Process:  
> 1.  solution name 입력 
> 2.  description & wrangler code 를 solution_metadata 에 삽입
> 3.  contents_type 을 solution_metadata 에 삽입
> 4.  description 의 solution icon 을 solution_metadata 에 삽입
> 5.  train 용 user parameter 를 solution_metadata 에 삽입
> 6.  s3 에 train data & artifacts 업로드 
> 7.  train 용 ecr repository 생성 및 docker container 업로드 
> 8.  inference 용 user parameter 를 solution_metadata 에 삽입
> 9.  s3 에 inference data & artifacts 업로드 
> 10. inference 용 ecr repository 생성 및 docker container 업로드 
> 11. solution 등록 



In [218]:
import sys
from IPython.display import SVG, display, HTML
import os 
from solution_register import SolutionRegister

try:
    del sys.modules['solution_register'] 
except:
    pass

register = SolutionRegister(
    infra_setup=infra_setup, 
    solution_info=solution_info)

register.debugging = False  ## default: False
register.skip_generation_docker = False  ## default: False
register.run(username=username, password=password)


#########################################
#######    Initiate ALO operation mode
#########################################

[SYSTEM] Solutoin 등록에 필요한 setup file 들을 load 합니다. 
[SYSTEM] infra_setup: 
{'AIC_URI': 'https://web.aic-dev.lgebigdata.com/',
 'BUILDAH_TAGS': ['Key=Company,Value=LGE',
                  'Key=Owner,Value=IC360',
                  'Key=HQ,Value=CDO',
                  'Key=Division,Value=CDO',
                  'Key=Infra Region,Value=KIC',
                  'Key=Service Mode,Value=OP',
                  'Key=Cost Type,Value=COMPUTING',
                  'Key=Project,Value=CIS',
                  'Key=Sub Project,Value=CISM',
                  'Key=System,Value=AIDX'],
 'BUILD_METHOD': 'docker',
 'ECR_TAG': 'latest',
 'LOGIN_MODE': 'static',
 'REGION': 'ap-northeast-2',
 'SOLUTION_TYPE': 'private',
 'VERSION': 1.0,
 'WORKSPACE_NAME': 'cism-ws'}

API setup 파일이 존재 하지 않으므로, Default 파일을 load 합니다. (path: ./api_setup.yaml)
[SYSTEM] solution_info.: 
{'contents_type': {'in

해당 계정으로 접근 가능한 workspace list: ['cism-ws']
[SYSTEM] 접근 요청하신 workspace (cism-ws) 은 해당 계정으로 접근 가능합니다.

#########################################
#######    Solution Name Creation
#########################################

<Response [200]>
[SUCCESS] 입력하신 Solution Name (tcr-test-ssh)은 사용 가능합니다. 
[SYSTEM] Solution Name List (in-use):
╒════╤══════════════════════════╕
│    │ AI solutions             │
╞════╪══════════════════════════╡
│  0 │ solution-hyunsoo-cism-20 │
├────┼──────────────────────────┤
│  1 │ pytest-solution-1        │
├────┼──────────────────────────┤
│  2 │ solution-hyunsoo-cism-2  │
├────┼──────────────────────────┤
│  3 │ solution-hyunsoo-cism-1  │
╘════╧══════════════════════════╛

#########################################
#######    Check ECR & S3 Resource
#########################################

[SYSTEM] AWS ECR:  
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/cism/
[SYSTEM] AWS S3 buckeet:  
s3-an2-hyunsoo-dev-cism

########

WARNING! Your password will be stored unencrypted in /home/sehyun.song/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[SYSTEM] AWS ECR | docker login result:
Login Succeeded

[SYSTEM] Target AWS ECR repository:
ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh/train/tcr-test-ssh
[SYSTEM] AWS ECR create-repository response: 
{'repository': {'repositoryArn': 'arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh/train/tcr-test-ssh', 'registryId': '086558720570', 'repositoryName': 'ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh/train/tcr-test-ssh', 'repositoryUri': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh/train/tcr-test-ssh', 'createdAt': datetime.datetime(2024, 1, 20, 14, 37, 22, 99000, tzinfo=tzlocal()), 'imageTagMutability': 'MUTABLE', 'imageScanningConfiguration': {'scanOnPush': False}, 'encryptionConfiguration': {'encryptionType': 'AES256'}}, 'ResponseMetadata': {'RequestId': '51cb98e3-b32a-4c85-bde5-6230a9bd92f1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-reques

WARNING! Your password will be stored unencrypted in /home/sehyun.song/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[SYSTEM] AWS ECR | docker login result:
Login Succeeded

[SYSTEM] Target AWS ECR repository:
ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh/inference/tcr-test-ssh
[SYSTEM] AWS ECR create-repository response: 
{'repository': {'repositoryArn': 'arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh/inference/tcr-test-ssh', 'registryId': '086558720570', 'repositoryName': 'ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh/inference/tcr-test-ssh', 'repositoryUri': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/tcr-test-ssh/inference/tcr-test-ssh', 'createdAt': datetime.datetime(2024, 1, 20, 14, 37, 52, 407000, tzinfo=tzlocal()), 'imageTagMutability': 'MUTABLE', 'imageScanningConfiguration': {'scanOnPush': False}, 'encryptionConfiguration': {'encryptionType': 'AES256'}}, 'ResponseMetadata': {'RequestId': '50d29041-e424-4a8e-a039-293ae1394b92', 'HTTPStatusCode': 200, 'HTTPHeaders'

##### 등록된 AI Solution 테스트 하기

In [251]:
try:
    del sys.modules['solution_register'] 
except:
    pass

from solution_register import SolutionRegister

register2 = SolutionRegister(
    infra_setup=infra_setup, 
    solution_info=solution_info)
    
register2.login(username, password)
# register2.register_solution_instance()
# register2.load_solution_instance_list()
# register2.register_stream()
# register2.request_run_stream()
status = register2.get_stream_status()
print(status)

register2.download_artifacts()


######################################################
#######    Initiate ALO operation mode
########################################################

[SYSTEM] Solutoin 등록에 필요한 setup file 들을 load 합니다. 
[SYSTEM] infra_setup: 
{'AIC_URI': 'https://web.aic-dev.lgebigdata.com/',
 'BUILDAH_TAGS': ['Key=Company,Value=LGE',
                  'Key=Owner,Value=IC360',
                  'Key=HQ,Value=CDO',
                  'Key=Division,Value=CDO',
                  'Key=Infra Region,Value=KIC',
                  'Key=Service Mode,Value=OP',
                  'Key=Cost Type,Value=COMPUTING',
                  'Key=Project,Value=CIS',
                  'Key=Sub Project,Value=CISM',
                  'Key=System,Value=AIDX'],
 'BUILD_METHOD': 'docker',
 'ECR_TAG': 'latest',
 'LOGIN_MODE': 'static',
 'REGION': 'ap-northeast-2',
 'SOLUTION_TYPE': 'private',
 'VERSION': 1.0,
 'WORKSPACE_NAME': 'cism-ws'}

API setup 파일이 존재 하지 않으므로, Default 파일을 load 합니다. (path: ./api_setup.yaml)
[SYSTEM] solution_in

In [ ]:

# 가상환경이 잘 connected 돼있는지 확인 
!which python 
# 현재 작업경로 확인 
!pwd

# s3로부터 다운로드받은 train artifacts를 scripts 폴더 상위 경로의 main.py랑 같은 위치로 옮기고 추론 실행 
import os
os.makedirs("./.train_artifacts",  exist_ok=True)

!tar -xvf ./train_artifacts.tar.gz -C ./.train_artifacts/
!cp -r .train_artifacts ../../
!rm -rf ./.train_artifacts

!python ../../main.py --mode inference